In [1]:
from DataLoader import DataLoader
from Anonymisation import Anonymisation
from Consistenter import Consistenter
from GUM import GraduallyUpdateMethod
from PostProcessor import RecordPostprocessor

In [2]:
dl = DataLoader('UCIMLAdult/uciml_adult.csv', 'UCIMLAdult/data.yaml', 'UCIMLAdult/column_info.json', 'UCIMLAdult/loading_data.json')
dl.data_loader()
dl.all_indifs(dl.private_data)
pass

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,3,1,0,2,1,4,0,1,1,0,4,1,1
1,4,2,0,0,2,3,0,1,0,0,2,1,1
2,3,3,1,1,3,4,0,1,0,0,4,1,1
3,4,3,2,0,3,3,4,1,0,0,4,1,1
4,2,3,0,0,4,1,4,0,0,0,4,2,1


In [3]:
anon = Anonymisation(10,3e-11)
anon.anonymiser(dl)
pass

epsilon: 10, delta: 3e-11, rho: 0.8616407135665874
one-way sigma: 8.685474957955632
indif sigma: 2.3399999999999998e-09
multi-way sigma: 5.838830013637393


In [4]:
c = Consistenter(anon, dl.all_attrs)
c.make_consistent(iterations = 5)
pass

Iteration 1 of 5 to make the marginals consistent and normalise them
Iteration 2 of 5 to make the marginals consistent and normalise them
Iteration 3 of 5 to make the marginals consistent and normalise them
Iteration 4 of 5 to make the marginals consistent and normalise them
Iteration 5 of 5 to make the marginals consistent and normalise them


In [5]:
gum = GraduallyUpdateMethod(dl, c)
gum.initialiser(view_iterations = 100)
syn_data = gum.synthesize(iterations = 100, num_records = int(c.num_synthesize_records))
pass

Iteration 1 of 100 completed to consist the marginal views
Iteration 2 of 100 completed to consist the marginal views
Iteration 3 of 100 completed to consist the marginal views
Iteration 4 of 100 completed to consist the marginal views
Iteration 5 of 100 completed to consist the marginal views
Iteration 6 of 100 completed to consist the marginal views
Iteration 7 of 100 completed to consist the marginal views
Iteration 8 of 100 completed to consist the marginal views
Iteration 9 of 100 completed to consist the marginal views
Iteration 10 of 100 completed to consist the marginal views
Iteration 11 of 100 completed to consist the marginal views
Iteration 12 of 100 completed to consist the marginal views
Iteration 13 of 100 completed to consist the marginal views
Iteration 14 of 100 completed to consist the marginal views
Iteration 15 of 100 completed to consist the marginal views
Iteration 16 of 100 completed to consist the marginal views
Iteration 17 of 100 completed to consist the marg

In [6]:
processor_priv = RecordPostprocessor(dl.private_data, dl.configpath, dl.datainfopath, dl.decode_mapping)
processor_pub = RecordPostprocessor(syn_data, dl.configpath, dl.datainfopath, dl.decode_mapping)
priv_data = processor_priv.post_process()
pub_data = processor_pub.post_process()

## Marginal Difference Score

In [7]:
t = 0
for i in range(len(dl.all_attrs)):
    for j in range(i+1, len(dl.all_attrs)):
        attr1 = dl.all_attrs[i]
        attr2 = dl.all_attrs[j]
        priv_marg = dl.two_way_marginal(attr1, attr2, dl.private_data)/len(dl.private_data)
        pub_marg = dl.two_way_marginal(attr1, attr2, syn_data)/len(syn_data)
        t += (priv_marg - pub_marg).abs().sum().sum()
t/(len(dl.all_attrs)*(len(dl.all_attrs)-1))

0.12219046379160214

## Classification Score

In [8]:
from sklearn import svm
import pandas as pd

/opt/anaconda3/envs/privsyntest/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
priv_x = priv_data.copy(deep = True).drop(columns = ['income'])
priv_x_ohe = pd.get_dummies(priv_x)
priv_y = priv_data['income']

In [10]:
pub_x = pub_data.copy(deep = True).drop(columns = ['income'])
pub_x_ohe = pd.get_dummies(pub_x)
for col in priv_x_ohe.columns:
    if col not in pub_x_ohe.columns:
        pub_x_ohe[col]=0
cols = priv_x_ohe.columns
pub_x_ohe = pub_x_ohe[cols]
pub_y = pub_data['income']

In [11]:
priv_classifier = svm.SVC()
priv_classifier.fit(priv_x_ohe, priv_y)
priv_score = priv_classifier.score(priv_x_ohe, priv_y)
priv_misscore = 1-priv_score
priv_misscore

0.19839482412677611

In [12]:
pub_classifier = svm.SVC()
pub_classifier.fit(pub_x_ohe, pub_y)
pub_score = pub_classifier.score(priv_x_ohe, priv_y)
pub_misscore = 1-pub_score
pub_misscore

0.2392817656934605